In [39]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor,  plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,  f1_score, r2_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from tqdm import tqdm 
from sklearn.ensemble import GradientBoostingClassifier,GradientBoostingRegressor
from sklearn.compose import make_column_transformer, make_column_selector
import numpy as np
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, LGBMClassifier
from catboost import CatBoostRegressor, CatBoostClassifier

In [40]:
df = pd.read_csv('HR_comma_sep.csv')
df

satisfaction_level  last_evaluation  number_project  \
0                    0.38             0.53               2   
1                    0.80             0.86               5   
2                    0.10             0.77               6   
3                    0.92             0.85               5   
4                    0.89             1.00               5   
...                   ...              ...             ...   
14990                0.40             0.57               2   
14991                0.37             0.48               2   
14992                0.37             0.53               2   
14993                0.11             0.96               6   
14994                0.37             0.52               2   

       average_montly_hours  time_spend_company  Work_accident  left  \
0                       157                   3              0     1   
1                       262                   6              0     1   
2                       247                   4              0     1   
3                       259                   5              0     1   
4                       224                   5              0     1   
...                     ...                 ...            ...   ...   
14990                   151                   3              0     1   
14991                   160                   3              0     1   
14992                   143                   3              0     1   
14993                   280                   4              0     1   
14994                   158                   3              0     1   

       promotion_last_5years Department  salary  
0                          0      sales     low  
1                          0      sales  medium  
2                          0      sales     low  
3                          0      sales     low  
4                          0      sales     low  
...                      ...        ...     ...  
14990                      0    support     low  
14991                      0    support     low  
14992                      0    support     low  
14993                      0    support     low  
14994                      0    support     low  

[14995 rows x 10 columns]

In [41]:
X , y = df.drop('left', axis=1) , df['left']

In [42]:
X_train, X_test, Y_train, Y_test = train_test_split(X,y,test_size=0.3, random_state=24)
X_train.dtypes

satisfaction_level       float64
last_evaluation          float64
number_project             int64
average_montly_hours       int64
time_spend_company         int64
Work_accident              int64
promotion_last_5years      int64
Department                object
salary                    object
dtype: object

In [43]:
ohe = OneHotEncoder(sparse_output=False , drop= 'first', handle_unknown='ignore').set_output(transform='pandas')
ct = make_column_transformer((ohe , make_column_selector(dtype_include=object)) , 
                              ('passthrough' , make_column_selector(dtype_exclude=object)),
                             verbose_feature_names_out=False).set_output(transform='pandas')

X_trn = ct.fit_transform(X_train)
X_tst = ct.transform(X_test)



In [44]:
# With XGboost 

n_trees = [25,50,100]
l_rate = [0.1,0.4,0.6,1]
depth = [3,5,None]
scores = []

for i in tqdm(range(len(n_trees))):
    for j in l_rate:
        for n in depth:
            xg = XGBClassifier(random_state=24,max_depth=n,learning_rate=j,n_estimators=n_trees[i])
            xg.fit(X_trn , Y_train)
            y_pred = xg.predict(X_tst)
            scores.append([n_trees[i],j,n, f1_score(Y_test, y_pred , pos_label=1)])

df1 = pd.DataFrame(scores , columns = [ 'Trees' , 'LR' , 'Depth' , 'F1'])
df1.sort_values('F1', ascending=False)    
            

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:02<00:00,  1.06it/s]


Trees   LR  Depth        F1
23     50  1.0    NaN  0.976008
29    100  0.4    NaN  0.975962
34    100  1.0    5.0  0.975586
35    100  1.0    NaN  0.975540
32    100  0.6    NaN  0.975516
31    100  0.6    5.0  0.973103
20     50  0.6    NaN  0.973077
28    100  0.4    5.0  0.971648
22     50  1.0    5.0  0.971319
19     50  0.6    5.0  0.969639
17     50  0.4    NaN  0.969609
33    100  1.0    3.0  0.969319
8      25  0.6    NaN  0.968085
11     25  1.0    NaN  0.967835
16     50  0.4    5.0  0.967054
10     25  1.0    5.0  0.966779
5      25  0.4    NaN  0.964580
7      25  0.6    5.0  0.963645
30    100  0.6    3.0  0.963285
21     50  1.0    3.0  0.962428
26    100  0.1    NaN  0.960584
4      25  0.4    5.0  0.958557
25    100  0.1    5.0  0.957623
27    100  0.4    3.0  0.957406
9      25  1.0    3.0  0.956438
14     50  0.1    NaN  0.955969
15     50  0.4    3.0  0.955889
18     50  0.6    3.0  0.955804
13     50  0.1    5.0  0.954102
6      25  0.6    3.0  0.953533
2      25  0.1    NaN  0.952941
24    100  0.1    3.0  0.950995
1      25  0.1    5.0  0.949535
3      25  0.4    3.0  0.949515
12     50  0.1    3.0  0.936893
0      25  0.1    3.0  0.931268

In [49]:
# With XGboost 

n_trees = [25,50,100]
l_rate = [0.1,0.4,0.6,1]
depth = [3,5,None]
scores = []

for i in tqdm(range(len(n_trees))):
    for j in l_rate:
        for n in depth:
            xg = (random_state=24,max_depth=n,learning_rate=j,n_estimators=n_trees[i])
            xg.fit(X_trn , Y_train)
            y_pred = xg.predict(X_tst)
            scores.append([n_trees[i],j,n, f1_score(Y_test, y_pred , pos_label=1)])

df1 = pd.DataFrame(scores , columns = [ 'Trees' , 'LR' , 'Depth' , 'F1'])
df1.sort_values('F1', ascending=False)    

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (4116266357.py, line 11)

In [ ]:
# With LGBM 

n_trees = [25,50,100]
l_rate = [0.1,0.4,0.6,1]
depth = [3,5,None]
scores = []

for i in tqdm(range(len(n_trees))):
    for j in l_rate:
        for n in depth:
            xg = LGBMClassifier(random_state=24,max_depth=n,learning_rate=j,n_estimators=n_trees[i])
            xg.fit(X_trn , Y_train)
            y_pred = xg.predict(X_tst)
            scores.append([n_trees[i],j,n, f1_score(Y_test, y_pred , pos_label=1)])

df1 = pd.DataFrame(scores , columns = [ 'Trees' , 'LR' , 'Depth' , 'F1'])
df1.sort_values('F1', ascending=False)    

In [ ]:
# With CatBoost

n_trees = [25,50,100]
l_rate = [0.1,0.4,0.6,1]
depth = [3,5,None]
scores = []

for i in tqdm(range(len(n_trees))):
    for j in l_rate:
        for n in depth:
            xg = CatBoostClassifier(random_state=24,max_depth=n,learning_rate=j,n_estimators=n_trees[i])
            xg.fit(X_trn , Y_train)
            y_pred = xg.predict(X_tst)
            scores.append([n_trees[i],j,n, f1_score(Y_test, y_pred , pos_label=1)])

df1 = pd.DataFrame(scores , columns = [ 'Trees' , 'LR' , 'Depth' , 'F1'])
df1.sort_values('F1', ascending=False)    